In [1]:
import numpy as np
import scipy.io
mat = scipy.io.loadmat('dataset_connected_NYC.mat') 

In [2]:
friendship_new = mat['friendship_new']
friendship_old = mat['friendship_old']#new_friendship_old
selected_checkins = mat['selected_checkins'] #new_checkins
selected_users_IDs = mat['selected_users_IDs'] #new_users_IDs

In [3]:
# 1. rebuild node index
offset1 = max(selected_checkins[:,0]);
dumy, dumy, n = np.unique(selected_checkins[:,1],return_index=True,return_inverse=True, axis=0)


selected_checkins[:,1] = n+1#+offset1; #n is the indices of the unique values in selected_checkins[:,1]
offset2 = max(selected_checkins[:,1]);
dumy, dumy, n = np.unique(selected_checkins[:,2],return_index=True,return_inverse=True, axis=0)

selected_checkins[:,2] = n+1#+offset2;
offset3 = max(selected_checkins[:,2]);
dumy, dumy, n = np.unique(selected_checkins[:,3],return_index=True,return_inverse=True, axis=0)
selected_checkins[:,3] = n+1#+offset3;

num_node_total = max(map(max, selected_checkins)) #max of the entire matrix = 8117

In [4]:
#verify sequence
selected_checkins-=1
print(selected_checkins.shape)
print(min(selected_checkins[:,0]))
print(max(selected_checkins[:,0]))
print(min(selected_checkins[:,1]))
print(max(selected_checkins[:,1]))
print(min(selected_checkins[:,2]))
print(max(selected_checkins[:,2]))
print(min(selected_checkins[:,3]))
print(max(selected_checkins[:,3]))


(105961, 4)
0
4023
0
167
0
3627
0
299


In [5]:
num_of_users = max(selected_checkins[:,0])
num_of_time = max(selected_checkins[:,1])
num_of_venues = max(selected_checkins[:,2])
num_of_categories = max(selected_checkins[:,3])

In [6]:
print(num_of_users)
print(num_of_time)
print(num_of_venues)
print(num_of_categories)

4023
167
3627
299


In [7]:
#separating training and testing data
train_size = int((80/100)*np.size(selected_checkins,0))
train_data = selected_checkins[:train_size]
nums_type = np.array([num_of_users+1, num_of_time+1, num_of_venues+1, num_of_categories+1])
test_data = selected_checkins[train_size:]
#data = {'train_data':train_data,'test_data':test_data,'num_types':num_types}

In [8]:
np.savez_compressed('lbsndata/train_data.npz', train_data = train_data, nums_type = nums_type)
np.savez_compressed('lbsndata/test_data.npz', test_data = test_data, nums_type = nums_type)


In [9]:
print(train_data.shape)

(84768, 4)


In [10]:
#selected_checkins format -
#selected_checkins (4 columns): user_index, time (hour in a week), venue_index, venue_category_index
#feeding it as it is to DHNE

%run hypergraph_embedding.py --data_path lbsndata --save_path lbsnresult -s 100 100 100 100 -e 50

Using TensorFlow backend.


this is args
Namespace(alpha=1, batch_size=16, data_path='lbsndata', embedding_size=[100, 100, 100, 100], epochs_to_train=50, hidden_size=64, learning_rate=0.01, num_neg_samples=5, options=None, prefix_path='model', save_path='lbsnresult', seed=None)
train_data.edge [[1800   38  983   61]
 [ 104   38  395   27]
 [ 570   38  343  234]
 ...
 [1106  139  368  153]
 [2514  139 2174  150]
 [2668  139 3077  292]]
train_data.nums_type [4024  168 3628  300]
4023 , 4024
167 , 168
3613 , 3628
297 , 300
Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_0 (InputLayer)            (None, 4096)         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (N

5298/5298 [==============================] - 648s 122ms/step - loss: 0.4119 - decode_0_loss: 1.8221e-04 - decode_1_loss: 5.4831e-04 - decode_2_loss: 1.0169e-04 - decode_3_loss: 2.0739e-04 - classify_layer_loss: 0.4108 - decode_0_mean_squared_error: 0.3341 - decode_1_mean_squared_error: 0.4283 - decode_2_mean_squared_error: 0.4937 - decode_3_mean_squared_error: 0.5204 - classify_layer_acc: 0.8400 - val_loss: 0.4251 - val_decode_0_loss: 1.6977e-04 - val_decode_1_loss: 6.4550e-04 - val_decode_2_loss: 1.6158e-04 - val_decode_3_loss: 1.5197e-04 - val_classify_layer_loss: 0.4239 - val_decode_0_mean_squared_error: 0.3075 - val_decode_1_mean_squared_error: 0.4492 - val_decode_2_mean_squared_error: 0.4814 - val_decode_3_mean_squared_error: 0.5123 - val_classify_layer_acc: 0.8333
Epoch 6/50
5298/5298 [==============================] - 652s 123ms/step - loss: 0.4063 - decode_0_loss: 1.8173e-04 - decode_1_loss: 4.4197e-04 - decode_2_loss: 9.7382e-05 - decode_3_loss: 1.9304e-04 - classify_layer_los

5298/5298 [==============================] - 639s 121ms/step - loss: 0.3957 - decode_0_loss: 1.7699e-04 - decode_1_loss: 3.0450e-04 - decode_2_loss: 8.7560e-05 - decode_3_loss: 1.6520e-04 - classify_layer_loss: 0.3949 - decode_0_mean_squared_error: 0.3424 - decode_1_mean_squared_error: 0.3892 - decode_2_mean_squared_error: 0.4875 - decode_3_mean_squared_error: 0.4455 - classify_layer_acc: 0.8465 - val_loss: 0.4233 - val_decode_0_loss: 1.2368e-04 - val_decode_1_loss: 3.9624e-04 - val_decode_2_loss: 1.9496e-04 - val_decode_3_loss: 2.8315e-04 - val_classify_layer_loss: 0.4223 - val_decode_0_mean_squared_error: 0.3198 - val_decode_1_mean_squared_error: 0.3541 - val_decode_2_mean_squared_error: 0.4728 - val_decode_3_mean_squared_error: 0.4518 - val_classify_layer_acc: 0.8333
Epoch 16/50
5298/5298 [==============================] - 631s 119ms/step - loss: 0.3964 - decode_0_loss: 1.7182e-04 - decode_1_loss: 3.0195e-04 - decode_2_loss: 8.5961e-05 - decode_3_loss: 1.6513e-04 - classify_layer_lo

5298/5298 [==============================] - 550s 104ms/step - loss: 0.4004 - decode_0_loss: 1.5813e-04 - decode_1_loss: 2.9657e-04 - decode_2_loss: 8.4481e-05 - decode_3_loss: 1.6623e-04 - classify_layer_loss: 0.3997 - decode_0_mean_squared_error: 0.3394 - decode_1_mean_squared_error: 0.3956 - decode_2_mean_squared_error: 0.4882 - decode_3_mean_squared_error: 0.5299 - classify_layer_acc: 0.8478 - val_loss: 0.3329 - val_decode_0_loss: 2.3359e-04 - val_decode_1_loss: 2.5190e-04 - val_decode_2_loss: 1.1165e-04 - val_decode_3_loss: 2.4743e-04 - val_classify_layer_loss: 0.3320 - val_decode_0_mean_squared_error: 0.3710 - val_decode_1_mean_squared_error: 0.4058 - val_decode_2_mean_squared_error: 0.4718 - val_decode_3_mean_squared_error: 0.5369 - val_classify_layer_acc: 0.8750
Epoch 26/50
5298/5298 [==============================] - 558s 105ms/step - loss: 0.4007 - decode_0_loss: 1.5357e-04 - decode_1_loss: 2.9810e-04 - decode_2_loss: 8.3596e-05 - decode_3_loss: 1.6564e-04 - classify_layer_lo

5298/5298 [==============================] - 549s 104ms/step - loss: 0.3889 - decode_0_loss: 1.4381e-04 - decode_1_loss: 2.5920e-04 - decode_2_loss: 8.3163e-05 - decode_3_loss: 1.7730e-04 - classify_layer_loss: 0.3882 - decode_0_mean_squared_error: 0.3386 - decode_1_mean_squared_error: 0.3882 - decode_2_mean_squared_error: 0.4639 - decode_3_mean_squared_error: 0.4212 - classify_layer_acc: 0.8495 - val_loss: 0.4728 - val_decode_0_loss: 1.3905e-04 - val_decode_1_loss: 2.0832e-04 - val_decode_2_loss: 3.1599e-05 - val_decode_3_loss: 8.2715e-05 - val_classify_layer_loss: 0.4723 - val_decode_0_mean_squared_error: 0.3299 - val_decode_1_mean_squared_error: 0.3606 - val_decode_2_mean_squared_error: 0.4871 - val_decode_3_mean_squared_error: 0.3980 - val_classify_layer_acc: 0.8333
Epoch 36/50
5298/5298 [==============================] - 549s 104ms/step - loss: 0.3920 - decode_0_loss: 1.3738e-04 - decode_1_loss: 2.5869e-04 - decode_2_loss: 8.3973e-05 - decode_3_loss: 1.7631e-04 - classify_layer_lo

5298/5298 [==============================] - 551s 104ms/step - loss: 0.3923 - decode_0_loss: 1.4040e-04 - decode_1_loss: 2.6213e-04 - decode_2_loss: 8.1075e-05 - decode_3_loss: 1.7017e-04 - classify_layer_loss: 0.3917 - decode_0_mean_squared_error: 0.3422 - decode_1_mean_squared_error: 0.4205 - decode_2_mean_squared_error: 0.4505 - decode_3_mean_squared_error: 0.5382 - classify_layer_acc: 0.8503 - val_loss: 0.3758 - val_decode_0_loss: 1.1094e-04 - val_decode_1_loss: 1.8881e-04 - val_decode_2_loss: 8.4471e-05 - val_decode_3_loss: 1.8734e-04 - val_classify_layer_loss: 0.3752 - val_decode_0_mean_squared_error: 0.3401 - val_decode_1_mean_squared_error: 0.4447 - val_decode_2_mean_squared_error: 0.4309 - val_decode_3_mean_squared_error: 0.5049 - val_classify_layer_acc: 0.8333
Epoch 46/50
5298/5298 [==============================] - 550s 104ms/step - loss: 0.3921 - decode_0_loss: 1.3845e-04 - decode_1_loss: 2.6965e-04 - decode_2_loss: 8.2095e-05 - decode_3_loss: 1.6843e-04 - classify_layer_lo

In [11]:
#embeddings are stored in lbsnresult/model_(no.ofemneddings)/embeddings.npy
#corresponding model stored in lbsnresult/model_(no.ofemneddings)/model.h5

embed = np.load('lbsnresult/model_100/embeddings.npy')
print(embed.shape)
print('embed[0].shape',embed[0].shape)
print('embed[1].shape',embed[1].shape)
print('embed[2].shape',embed[2].shape)
print('embed[3].shape',embed[3].shape)

(4,)
embed[0].shape (4024, 100)
embed[1].shape (168, 100)
embed[2].shape (3628, 100)
embed[3].shape (300, 100)


In [29]:
#location prediction on the learnt embeddings
#selected_checkins (4 columns): user_index, time (hour in a week), venue_index, venue_category_index

from scipy.spatial import distance

count = 0

for d in range(selected_checkins.shape[0]):
    test = selected_checkins[d]
    user = test[0]
    time = test[1]
    dist = []
    
    for i in range(min(selected_checkins[:,2]),max(selected_checkins[:,2])+1):
        summ = (1 - distance.cosine(embed[0][user],embed[2][i]))+(1 - distance.cosine(embed[0][time],embed[2][i]))
        dist.append((abs(summ),i))
        
    dist = sorted(dist, key=lambda student: student[0])
    if test[2] in (np.array(dist[0:10])[:,1]):
        count+=1
print("this is count "+str(count))
print("this is count fraction "+str(count/selected_checkins.shape[0]))

KeyboardInterrupt: 